In [1]:
import os
#os.chdir("/home/jovyan/work/carlos/complete_execution_andalucia")
import pandas as pd
import numpy as np
os.chdir("./../")
#from src.VariableNameConversion import VariableNameConversion
#os.chdir("./notebooks/")

In [ ]:
# Region: Lublin
NUTS2_code = "PL81"

os.getcwd()


In [ ]:
microdata_path = "./data/use_case_poland/microdata"

microdata = pd.DataFrame()

for file in sorted(os.listdir(microdata_path)):
    if file != "POL2030.csv" and not file.startswith("PL"):
        print(file)
        df_ = pd.read_csv(os.path.join(microdata_path, file))

        if file in ["POL2014.csv", "POL2015.csv", "POL2016.csv"]:
            nuts2_code_ = "PL31"
        else:
            nuts2_code_ = NUTS2_code

        # Compose an unique df with all the years
        microdata = pd.concat([microdata, df_])

        print(nuts2_code_)

        # Save only required NUTS
        selection = df_[df_["A_LO_40_N2"]==nuts2_code_]
        
        selection["A_LO_40_N2"] = selection["A_LO_40_N2"].apply(lambda x: NUTS2_code)
        print(selection.shape)
        
        selection.to_csv(f'./data/use_case_poland/microdata/{file.replace("POL", "PL")}', index=False)
        
microdata["YEAR"] = 2030

microdata.reset_index(drop=True).to_csv("./data/use_case_poland/microdata/POL2030.csv", index=False)

In [ ]:
print(f'Minimum economic size: {microdata["SE005"].min()}')

In [ ]:
totals_by_crop_filepath = "./"# Set path

os.listdir(totals_by_crop_filepath)

In [ ]:
metadata = pd.read_csv(os.path.join(totals_by_crop_filepath, "Poland-crops-totals"), sep=",|\t")

metadata["geo\\TIME_PERIOD"] = metadata.apply(lambda x: NUTS2_code, axis=1)


metadata = metadata[metadata["geo\\TIME_PERIOD"].isin([NUTS2_code])]


#data = data[["crops", "unit", "geo\TIME_PERIOD", "2005 ", "2007 ", "2010 ", "2013 ", "2016 ", "2020", ]]
metadata = metadata[["crops", "unit", "2013 ", "2016 ", "2020"]].rename(columns={"2013 ": "2013", "2016 ": "2016"})


for c in ["2013", "2016", "2020"]:
    metadata[c] = metadata.apply(lambda x: x[c].replace(" ", "").replace(":", "").replace("c", "").replace("u", ""), axis=1)
    metadata[c] = metadata.apply(lambda x: "0" if x[c]=="" else x[c], axis=1)
    metadata[c] = metadata[c].astype(int)

metadata = metadata.groupby(by=["crops", "unit"]).sum().reset_index()

display(metadata)


In [ ]:
for crop in metadata["crops"].unique():
    print(f'"{crop}": ""')

In [ ]:
links = {
    #"UAA": "Utilised agricultural area",
    #"UAA09S": "Other utilised agricultural area n.e.c. - under glass or high accessible cover",
    #"UAAT": "Utilised agricultural area - outdoor",
    #"UAAS": "Utilised agricultural area - under glass or high accessible cover",
    #"ARA09S": "Other arable land crops n.e.c. - under glass or high accessible cover",
    #"ARA": "Arable land",
    #"C0000": "Cereals for the production of grain (including seed)",
    "C1110": "Common wheat and spelt",
    "C1120": "Durum wheat",
    "C1200": "Rye", #"Rye and winter cereal mixtures (maslin)",
    "C1300": "Barley",
    "C1400": "Oats", #"Oats and spring cereal mixtures (mixed grain other than maslin)",
    "C1500": "Grain maize", #"Grain maize and corn-cob-mix",
    "C1600_1700_1900": "Other cereals for the production of grain", #"Other cereals (including triticale and sorghum)",
    "C2000": "Rice",
    #"P0000": "Dry pulses and protein crops for the production of grain (including seed and mixtures of cereals and pulses)",
    "P1000": "Peas, field beans and sweet lupines", #"Field peas, beans and sweet lupins",
    #"P9000": "Other dry pulses and protein crops n.e.c.",
    #"R0000": "Root crops",
    "R1000": "Potatoes (including early potatoes and seed potatoes)", #"Potatoes (including seed potatoes)",
    "R2000": "Sugar beet (excluding seed)",
    "R9000": "Fodder roots and brassicas (excluding seed)", #"Other root crops n.e.c.",
    #"I0000": "Industrial crops",
    "I1110": "Rape and turnip rape", #"Rape and turnip rape seeds",
    "I1120": "Sunflower", #"Sunflower seed",
    "I1130": "Soya",
    "I1140": "Linseed (oil flax)", #"Linseed (oilflax)",
    "I1150_2300": "Cotton", #"Cotton seed and fibre",
    "I1190": "Other oil seed crops", #"Other oilseed crops n.e.c.",
    #"I2000": "Fibre crops",
    "I2100": "Fibre flax",
    "I2200": "Hemp",
    "I2900": "Other fibre plants", #"Other fibre crops n.e.c.",
    "I3000": "Tobacco",
    "I4000": "Hops",
    "I5000": "Aromatic plants, medical and culinary plants", #"Aromatic, medicinal and culinary plants",
    #"I6000_9000": "Other industrial crops not mentioned elsewhere", #"Other industrial crops including energy crops n.e.c.",
    "I9000": "Other industrial crops not mentioned elsewhere", #"Other industrial crops n.e.c.",
    #"G0000": "Plants harvested green from arable land",
    "G0000X1000": "Plants harvested green from arable land excluding temporary grasses and grazings",
    "G1000": "Temporary grasses and grazings",
    "G2000": "Leguminous plants", #"Leguminous plants harvested green",
    "G3000": "Green maize",
    "G9000": "Other plants harvested green from arable land",
    "G9100": "Other plants harvested green but not mentioned elsewhere", #"Other cereals harvested green (excluding green maize)",
    "G9900": "Other plants harvested green from arable land n.e.c.",
    #"V0000_S0000": "Fresh vegetables (including melons) and strawberries",
    #"V0000_S0000T": "Fresh vegetables (including melons) and strawberries - outdoor",
    "V0000_S0000TO": "Fresh vegetables, melons and strawberries -Open field", #"Fresh vegetables (including melons) and strawberries grown in rotation with non-horticultural crops (open field)",
    "V0000_S0000TK": "Fresh vegetables, melons and strawberries -Market gardening", #"Fresh vegetables (including melons) and strawberries grown in rotation with horticultural crops (market gardening)",
    "V0000_S0000S": "Fresh vegetables, melons and strawberries -Under glass or under other (accessible) protective cover.", #"Fresh vegetables (including melons) and strawberries - under glass or high accessible cover",
    #"N0000": "Flowers and ornamental plants (excluding nurseries)",
    "N0000T": "Flowers and ornamental plants -Outdoor or under low protective cover", #"Flowers and ornamental plants (excluding nurseries) - outdoor",
    "N0000S": "Flowers and ornamental plants -Under glass or under other protective cover", #"Flowers and ornamental plants (excluding nurseries) - under glass or high accessible cover",
    "E0000": "Arable land seed and seedling", #"Seeds and seedlings",
    "ARA99": "Other arable land crops n.e.c.",
    #"Q0000": "Fallow land",
    #"J0000": "Permanent grassland",
    "J1000": "Permanent pastures and meadows",
    "J2000": "Permanent rough grazings",
    "J3000TE": "Permanent agricultural grassland not in use, eligible for subsidies - outdoor",
    "PECR": "Permanent crops",
    "PECRT": "Permanent crops - outdoor",
    "PECRS": "Permanent crops under glass or high accessible cover",
    "F0000": "Fruits, berries and nuts (excluding citrus fruits, grapes and strawberries)",
    "F1000_2000": "Fruits from temperate, subtropical and tropical climate zones",
    "F1000": "Fruits from temperate climate zones",
    #"F1100": "Pome fruits",
    #"F1200": "Stone fruits",
    "F2000": "Fruits from subtropical and tropical climate zones",
    "F3000": "Berries (excluding strawberries)",
    "F4000": "Nuts",
    #"T0000": "Citrus fruits",
    "W1000": "Grapes",
    "W1100": "Grapes for wines",
    "W1110_1120": "Grapes for wines with geographical indication (PDO/PGI)",
    "W1110": "Grapes for wines with protected designation of origin (PDO)",
    "W1120": "Grapes for wines with protected geographical indication (PGI)",
    "W1190": "Grapes for other wines n.e.c. (without PDO/PGI)",
    "W1200": "Grapes for table use",
    "W1300": "Grapes for raisins",
    #"O1000": "Olives",
    "O1100": "Table olives", #"Olives for table use",
    "O1910": "Olives for oil production (sold in the form of fruit)", #"Olives for oil",
    "L0000": "Nurseries",
    "PECR9_H9000": "Other permanent crops including other permanent crops for human consumption",
    "X0000": "Other permanent crops, of which Christmas tress", #"Christmas trees",
    "U1000": "Mushrooms", #"Cultivated mushrooms",
    "SRCAA": "Short rotation coppice areas"
}

links_inv = dict(zip(links.values(), links.keys()))

print(links_inv)


#### Load FADN individual codes
Use Andalusia product mapping to load individual FADN codes and their description. 

In [ ]:
product_mapping_and = pd.read_csv("./data/use_case_andalusia/metadata/Product_Mapping.csv")

product_mapping = product_mapping_and.copy(deep=True)[["FADN Included products", "FADN Included products IDs"]].drop_duplicates()
product_mapping.at[24, "FADN Included products"] = "Fibre flax"
product_mapping.at[18, "FADN Included products"] = "Cotton"

product_mapping.iloc[[22, 23, 24]]


In [13]:
product_mapping["col"] = product_mapping.apply(lambda x: links_inv[x["FADN Included products"]] if x["FADN Included products"] in links_inv.keys() else np.nan, axis=1)


In [ ]:
for k in links.keys():
    if not k in product_mapping["col"].tolist():
        print(k, links[k])

In [18]:
# Select only linked FADN codes
product_mapping = product_mapping[~product_mapping["col"].isna()].reset_index(drop=True)

In [ ]:
fadn_codes_link = dict(zip(product_mapping["col"], product_mapping["FADN Included products IDs"], ))

print(fadn_codes_link)

### Split metadata into area and holdings

In [20]:
metadata_area = metadata[metadata["unit"]=="HA"].reset_index(drop=True)
metadata_holdings = metadata[metadata["unit"]=="HLD"].reset_index(drop=True)


#### Process area data

In [ ]:
# Change crop code: From Eurostat code -> FADN code
metadata_area["crops"] = metadata_area.apply(lambda x: fadn_codes_link[x["crops"]] if x["crops"] in fadn_codes_link.keys() else x["crops"], axis=1)

# Select only identified FADN codes
metadata_area = metadata_area[metadata_area.apply(lambda x: x["crops"] in product_mapping["FADN Included products IDs"].tolist(), axis=1)]

metadata_area = metadata_area.drop(columns=["unit"])

metadata_area

#### Process holdings data

In [ ]:
try:
    n_holdings_economic_size_filepath = "./../../../../../mnt/c/users/idener/downloads/Poland-n_holdings-economic_size"

    n_holdings_df = pd.read_csv(os.path.join(n_holdings_economic_size_filepath, 'Poland-n_holdings-economic_size'), sep=",|\t").rename(columns={"2013 ": "2013", "2016 ": "2016", })

    economic_size_link = {
        "TOTAL": "Total", 
        "KE4-7": "From 4 000 to 7 999 euros", 
        "KE8-14": "From 8 000 to 14 999 euros", 
        "KE15-24": "From 15 000 to 24 999 euros", 
        "KE25-49": "From 25 000 to 49 999 euros", 
        "KE50-99": "From 50 000 to 99 999 euros", 
        "KE100-249": "From 100 000 to 249 999 euros", 
        "KE250-499": "From 250 000 to 499 999 euros", 
        "KE_GE500": "500 000 euros or over", 
    }

    n_holdings_df["so_eur"] = n_holdings_df["so_eur"].apply(lambda x: economic_size_link[x])

    for year in ["2013", "2016", "2020", ]:
        n_holdings_df[year] = n_holdings_df.apply(lambda x: x[year].replace(" ", "").replace(":", "0"), axis=1)
        n_holdings_df[year] = n_holdings_df[year].astype(int)


    n_holdings_df["geo\\TIME_PERIOD"] = n_holdings_df["geo\\TIME_PERIOD"].apply(lambda x: x if x==NUTS2_code else NUTS2_code)

    n_holdings_df = n_holdings_df.groupby(by=["so_eur", "unit", "freq", "uaarea", "crops", "geo\TIME_PERIOD"]).agg({
        "2013": "sum", 
        "2016": "sum", 
        "2020": "sum"}
    ).reset_index()[["so_eur", "unit", "2013", "2016", "2020"]]


    n_holdings_df = n_holdings_df.melt(id_vars=["so_eur", "unit"],
                    value_vars=["2013", "2016", "2020"],
                    var_name="YEAR").pivot_table(values="value",
        index=["so_eur", "YEAR", ],
        columns="unit",).reset_index().rename(columns={
            "so_eur": 'Economic Size', 
            "HA": 'Hectare', 
            "HLD": 'Holding'
        })[["YEAR", "Economic Size", "Hectare", "Holding"]]

    n_holdings_df.to_csv("./data/use_case_poland/metadata/number_of_holdings.csv", index=False)

    display(n_holdings_df)

except:
    print("Error loading data from downloads")
    pass

In [ ]:
n_holdings_economic = pd.read_csv("./data/use_case_poland/metadata/number_of_holdings.csv")

n_holdings_economic

In [ ]:
n_farms_sample_2013 = n_holdings_economic[(n_holdings_economic["YEAR"]==2013)&(n_holdings_economic["Economic Size"]!="Total")]["Holding"].sum()
print(n_farms_sample_2013)

n_farms_sample_2016 = n_holdings_economic[(n_holdings_economic["YEAR"]==2016)&(n_holdings_economic["Economic Size"]!="Total")]["Holding"].sum()
print(n_farms_sample_2016)

n_farms_sample_2020 = n_holdings_economic[(n_holdings_economic["YEAR"]==2020)&(n_holdings_economic["Economic Size"]!="Total")]["Holding"].sum()
print(n_farms_sample_2020)


In [ ]:
n_farms_sample_2014 = int(0.66*n_farms_sample_2013 + 0.33*n_farms_sample_2016)
print(n_farms_sample_2014)

n_farms_sample_2018 = int((n_farms_sample_2016 + n_farms_sample_2020)/2)
print(n_farms_sample_2018)

In [ ]:
n_farms_total_2013 = n_holdings_economic[(n_holdings_economic["YEAR"]==2013)&(n_holdings_economic["Economic Size"]=="Total")]["Holding"].sum()
print(n_farms_total_2013)

n_farms_total_2016 = n_holdings_economic[(n_holdings_economic["YEAR"]==2016)&(n_holdings_economic["Economic Size"]=="Total")]["Holding"].sum()
print(n_farms_total_2016)

n_farms_total_2020 = n_holdings_economic[(n_holdings_economic["YEAR"]==2020)&(n_holdings_economic["Economic Size"]=="Total")]["Holding"].sum()
print(n_farms_total_2020)

In [ ]:
n_farms_total_2014 = int(0.66*n_farms_total_2013 + 0.33*n_farms_total_2016)
print(n_farms_total_2014)

n_farms_total_2018 = int((n_farms_total_2016 + n_farms_total_2020)/2)
print(n_farms_total_2018)

In [ ]:
metadata_area["2014"] = metadata_area.apply(lambda x: int(0.66*x["2013"]+0.33*x["2016"]), axis=1)
metadata_area["2018"] = metadata_area.apply(lambda x: x[["2016", "2020"]].mean(), axis=1)
metadata_area

#### Compute average land per crop and year

In [29]:
for year in ["2013", "2014", "2016", "2018", "2020"]:
    metadata_area[f"avg {year}"] = metadata_area.apply(lambda x: x[year]/eval(f"n_farms_sample_{year}"), axis=1)


## Load microdata

In [31]:
microdata_2014 = pd.read_csv("./data/use_case_poland/microdata/PL2014.csv")
microdata_2018 = pd.read_csv("./data/use_case_poland/microdata/PL2018.csv")


In [32]:
microdata_2014 = microdata_2014[microdata_2014["A_LO_40_N2"]==NUTS2_code].reset_index(drop=True)
microdata_2018 = microdata_2018[microdata_2018["A_LO_40_N2"]==NUTS2_code].reset_index(drop=True)

In [ ]:
microdata_2018[[c for c in microdata_2014.columns if c.endswith("TA")]].sum()


In [ ]:
total_area_variables = [var for var in [f"I_A_{code}_TA" for code in metadata_area["crops"].unique()] if var in microdata_2014.columns]

microdata_area_2014 = microdata_2014[total_area_variables].apply(lambda x: x/1, axis=1).copy(deep=True)
microdata_area_2018 = microdata_2018[total_area_variables].apply(lambda x: x/1, axis=1).copy(deep=True)

display(microdata_area_2014)
display(microdata_area_2018)


In [35]:
mu_totals_2014 = metadata_area[metadata_area["crops"].isin([int(var.replace("I_A_", "").replace("_TA", "")) for var in microdata_area_2014.columns])]["avg 2014"].to_numpy()
mu_totals_2018 = metadata_area[metadata_area["crops"].isin([int(var.replace("I_A_", "").replace("_TA", "")) for var in microdata_area_2018.columns])]["avg 2018"].to_numpy()


In [ ]:
pd.DataFrame([mu_totals_2014, mu_totals_2018]).transpose().apply(lambda x: np.abs(x[0] - x[1]), axis=1)


## Use empirical likelihood to compute weights

In [ ]:
from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage

pandas2ri.activate()

# Instantiate R as a python object
R = robjects.r

# Load R function from script
#with open("./src/R-scripts/R_functions.R") as r_file:
with open("./src/R-scripts/R_functions.R") as r_file:
    R_functions_string = r_file.read()

# Convert text to R code
R_functions = SignatureTranslatedAnonymousPackage(R_functions_string, "R_functions")


In [38]:
# Empirical likelihood ratio test for the means
# weights is composed from
#   {wts, nits}
weights_2014_ = R_functions.el_test_function(microdata_area_2014, mu_totals_2014)
weights_2014 = weights_2014_[5]

weights_2018_ = R_functions.el_test_function(microdata_area_2018, mu_totals_2018)
weights_2018 = weights_2018_[5]

In [ ]:
import matplotlib.pyplot as plt

plt.hist(weights_2014, bins=100)
plt.show()

plt.hist(weights_2018, bins=100)
plt.show()

In [40]:
# Compute normalized weights for microdata rows
# Use "wts" property from weights
weights_norm_2014 = R_functions.normalize_weights(weights_2014_)
weights_norm_2018 = R_functions.normalize_weights(weights_2018_)


In [41]:
df_weights_2014 = pd.DataFrame(weights_norm_2014, columns=["A_TY_80_W"]).apply(lambda x: x*n_farms_sample_2014)
df_weights_2018 = pd.DataFrame(weights_norm_2018, columns=["A_TY_80_W"]).apply(lambda x: x*n_farms_sample_2018)

In [43]:
sample_2014 = pd.concat([microdata_2014, df_weights_2014], axis=1)
sample_2014.to_csv(os.path.join(microdata_path, "PL2014.csv"), index=False)
sample_2018 = pd.concat([microdata_2018, df_weights_2018], axis=1)
sample_2018.to_csv(os.path.join(microdata_path, "PL2018.csv"), index=False)

#pd.concat([microdata_2014, df_weights_2014], axis=1).to_csv(os.path.join(microdata_path, "CM2014.csv"), index=False)
#pd.concat([microdata_2018, df_weights_2018], axis=1).to_csv(os.path.join(microdata_path, "CM2018.csv"), index=False)

In [ ]:
sample_2014[[c for c in sample_2014.columns if c.startswith("I_A_") and c.endswith("_TA")]].sum()

In [ ]:
pm = pd.read_csv("./data/use_case_poland/metadata/Product_Mapping.csv")

display(pm)

## Test Variable Name Conversion

In [46]:
from src.VariableNameConversion import VariableNameConversion

In [ ]:
requested_number_of_farms = 1000
USE_CASE = "poland"
BASE_PATH = f"./data/use_case_{USE_CASE}"
YEAR = "2014"
TOTALS_VARIABLES = ["cultivatedArea"]

vnc = VariableNameConversion(BASE_PATH, USE_CASE, YEAR, TOTALS_VARIABLES)

sp_end, _, _ = vnc.main()


## Load Crop Representativeness

In [48]:
representativeness_2014 = pd.read_csv("./data/use_case_poland/results/FADN_Representativeness_2014.csv")

pd.set_option('display.max_rows', 500)

In [ ]:
rep_2014 = representativeness_2014[representativeness_2014["fadn_code_organic"].apply(lambda x: not "ORG" in x)].sort_values(by=["total_area", "production_quantity"], ascending=False)

rep_2014 = rep_2014[rep_2014["n_appearances_abs"]>0]

rep_2014

In [ ]:
crops_grouping = {
    "CER": [10150, 10110, 10130, 10190, 10140, 
            10170, 10120], 
    "MAIZE": [
        10160, ],
    "BEET": [10400, ], 
    "OTHER": [10601, 10602, 11000, 10923, 
              10603, 10604, 10605, 10607, 10608, 10609, 10610, 10611, 10612, 10613, 10690, 10810, 10820, 10830, 10840, 10850, 11100, 11210, 11220, 11300, 20000, 40330, 40340, 40411, 40412, 40420, 40451, 40452, 40460, 40470, 40480, 40500, 40600, 40610, 40700, 40800, 50100, 50200, 50210, 50900, 60000, 90100, 90200, 90300, 90310, 90320, 90330, 90900, 
              10735, 10737, 10734, 10711, 10712, 10720, 10731, 10732, 10733, 10736, 10790, # From VEG
              ],
    "PROT": [10290, 10210, 10220, 
                10606, 10922],
    "GRAZING": [30100, 10921, 30200, 
                10500, 10910, 30300], 
    "FRUITS": [40111, 
               10738, 10739, 40112, 40113, 40114, 40115, 40120, 40130, 40210, 40220, 40230, 40290, 40310, 40320, 40430, 40440, ], 
    "POTATO": [10300, 
               10390, 10310 ],
    }

grouping = pd.DataFrame([(crop, group) for group in crops_grouping.keys() for crop in crops_grouping[group]], columns=["fadn_code", "product_group"])

grouping_dict = dict(zip(grouping["fadn_code"], grouping["product_group"]))
grouping_dict

In [53]:
product_mapping_pol = product_mapping_and.copy(deep=True)

In [ ]:
product_mapping_pol["production type"] = product_mapping_pol.apply(lambda x: "organic" if x["CUSTOM GROUP (EN)"].startswith("ORG") else "conventional", axis=1)
product_mapping_pol["CUSTOM GROUP (EN)"] = product_mapping_pol.apply(lambda x: grouping_dict[x["FADN Included products IDs"]], axis=1)
product_mapping_pol["CUSTOM GROUP (EN)"] = product_mapping_pol.apply(lambda x: x["CUSTOM GROUP (EN)"] if x["production type"]=="conventional" else f'ORG_{x["CUSTOM GROUP (EN)"]}', axis=1)

product_mapping_pol

In [55]:
product_mapping_pol.to_csv("./data/use_case_poland/metadata/Product_Mapping.csv", index=False)

### Product Groups

In [ ]:
product_groups_and = pd.read_csv("./data/use_case_andalusia/metadata/Product_Groups.csv")

display(product_groups_and)

In [ ]:
unique_categories = pd.DataFrame([cat for list_ in  product_groups_and["Categories"].unique().tolist() for cat in eval(list_)]).drop_duplicates()[0].tolist()

print(unique_categories)

In [ ]:
for p in product_mapping_pol["CUSTOM GROUP (EN)"].unique():
    print(f'"{p}": {unique_categories}, ')

In [59]:
product_groups_pol_dict = {
    "CER": ['Arable', 'Cereal', ], 
    "MAIZE": ['Arable', 'Cereal', ], 
    "LEGUMES": ['Arable', 'FixingNitrogen'], 
    "POTATO": ['Arable', ], 
    "BEET": ['Arable', ], 
    "GRAZING": ['Arable', 'LivestockFood', 'MeadowsAndPastures', ], 
    "OTHER": ['Arable', ], 
    "FRUITS": ['Perennial', ], 

    "ORG_CER": ['Arable', 'Cereal', ], 
    "ORG_MAIZE": ['Arable', 'Cereal', ], 
    "ORG_LEGUMES": ['Arable', 'FixingNitrogen'], 
    "ORG_POTATO": ['Arable', ], 
    "ORG_BEET": ['Arable', ], 
    "ORG_GRAZING": ['Arable', 'LivestockFood', 'MeadowsAndPastures', ], 
    "ORG_OTHER": ['Arable', ], 
    "ORG_FRUITS": ['Perennial', ], 

    "OTHER_LIVESTOCK":	['OTHER'], 
    "DAIRY":           ['Arable', 'MILK'], 
    }

In [ ]:
product_groups_pol = pd.DataFrame(zip(product_groups_pol_dict.keys(), product_groups_pol_dict.values()), columns=["PRODUCT GROUP", "Categories"])

product_groups_pol["Organic"] = product_groups_pol.apply(lambda x: 1 if x["PRODUCT GROUP"].startswith("ORG") else 0, axis=1)

product_groups_pol

In [61]:
product_groups_pol.to_csv("./data/use_case_poland/metadata/Product_Groups.csv", index=False)

In [1]:
import pandas as pd
import numpy as np
import os

os.chdir("./..")


In [2]:
dd = pd.read_csv("./data/use_case_poland/microdata/POL2015.csv")

for c in ["regionLevel1", "regionLevel1Name", "regionLevel2", "regionLevel2Name", "regionLevel3", "regionLevel3Name", ]:
    dd[c] = 0.0

In [3]:
dd["regionLevel1"] = "PL81"

## Region Level 3

In [18]:
dd__ = dd.copy(deep=True)

In [29]:
class RegionLevelPoland():
    def __init__(self, ):
        
        pass

    def _fill_regionLevel(self, sp):
        """
        Assign district value based on experimental likelihood.

        Parameters
        ----------
        sp: pd.DataFrame
            synthetic population

        Returns
        ----------
        sp: pd.DataFrame
            synthetic population with regionLevel and regionLevelName values filled
        """

        nuts_info = pd.DataFrame([
            ["NUTS2", "PL81",  "Lubelskie", 1, ],

            ["NUTS3", "PL811", "Bialski",            0.25, ], 
            ["NUTS3", "PL812", "Chełmsko-zamojski",  0.25, ], 
            ["NUTS3", "PL814", "Lubelski",           0.25, ], 
            ["NUTS3", "PL815", "Puławski",           0.25, ], 
            
            ], columns=["NUTS", "NUTS code", "NUTS name", "Number of holdings perc"])

        display(nuts_info)

        
        #n_farms_real = sample_2014["A_TY_80_W"].sum().astype(int)
        n_farms_real = sp.shape[0]
        
        # Total number of farms
        print(f'Total Number of farms according to microdata in year 2014: {n_farms_real}')

        nuts2_info = nuts_info[nuts_info["NUTS"]=="NUTS2"]
        nuts2_link_dict = dict(zip(nuts2_info["NUTS code"].tolist(), nuts2_info["NUTS name"].tolist(), ))

        nuts3_info = nuts_info[nuts_info["NUTS"]=="NUTS3"]
        nuts3_link_dict = dict(zip(nuts3_info["NUTS code"].tolist(), nuts3_info["NUTS name"].tolist(), ))

        values = nuts3_info["NUTS code"].tolist()
        probabilities = nuts3_info["Number of holdings perc"].tolist()

        # Assignation regionLevel1
        sp["regionLevel1Name"] = sp.apply(lambda x: nuts2_link_dict[x["regionLevel1"]], axis=1)

        # Assignation regionLevel2
        sp["regionLevel2"] = np.random.choice(values, n_farms_real, p=probabilities)
        sp["regionLevel2Name"] = sp.apply(lambda x: nuts3_link_dict[x["regionLevel2"]], axis=1)

        # Assignation regionLevel3
        # Divide data in 5
        n_rl3 = 5

        print(".-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-")
        for rl2 in sp["regionLevel2"].unique():
            indexes = sp[sp["regionLevel2"]==rl2].index

            values = [f"{rl2}{i}" for i in range(1, 1+n_rl3)]
            probabilities = [1/n_rl3]*n_rl3

            sp.loc[indexes, "regionLevel3"] = np.random.choice(values, len(indexes), p=probabilities)
            

        sp["regionLevel3Name"] = sp.apply(lambda x: x["regionLevel3"], axis=1)

        # regionLevel must be an integer
        for rl in [1, 2, 3]:
            sp[f"regionLevel{rl}"] = sp.apply(lambda x: int(x[f"regionLevel{rl}"].replace("PL", "")), axis=1)

        
        return sp
    


In [30]:
rlp = RegionLevelPoland()

In [ ]:
rr = rlp._fill_regionLevel(dd__)

In [ ]:
for c in ["regionLevel1", "regionLevel1Name", "regionLevel2", "regionLevel2Name", "regionLevel3", "regionLevel3Name", ]:
    print(c, rr[c].unique())